Jesus Ramirez Delgado
A01274723




IMPORTAR DATA SET (CODIGO PROPORCIONADO EN EL ENTREGABLE)

In [ ]:

import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

url = 'https://www.apradie.com/datos/dataus.csv'
response = requests.get(url, headers=headers)
with open('dataus.csv', 'wb') as f:
    f.write(response.content)

# Import as data frames
dataus = pd.read_csv('dataus.csv')

# Display the first few rows of each DataFrame to confirm
print("usfirms DataFrame:")
display(dataus.head())

usfirms DataFrame:


,firm,year,revenue,ebit,netincome,totalassets,totalliabilities,shortdebt,longdebt,retainedearnings,stockprice,marketvalue,epsp,industry
0,AAP,2001,2517639.0,98212.0,25005.0,1950615.0,1662044.0,58463.0,932022.0,-205294.0,14.405094,0.000,NaN,Trade
1,AAP,2002,3287883.0,201985.0,75967.0,1965225.0,1496869.0,11559.0,724832.0,-140275.0,14.158977,1745366.331,0.043525,Trade
2,AAP,2003,3493696.0,288234.0,172234.0,1983071.0,1351827.0,53305.0,422780.0,-15340.0,23.569340,3004390.809,0.057327,Trade
3,AAP,2004,3770297.0,328758.0,190968.0,2201962.0,1479647.0,51884.0,438300.0,172648.0,25.295056,3211784.154,0.059459,Trade
4,AAP,2005,4264971.0,408492.0,231910.0,2542149.0,1622378.0,82930.0,406040.0,407373.0,37.751481,4701384.545,0.049328,Trade


USO DE LLM´S

UTILICE CHAT GPT PARA AGILIZAR EL CODIGO PARA EL MODELO DEL ENTREGABLE

## CONOCER TIPOS DE INDUSTRIA

In [ ]:
print("Industrias únicas:")
print(dataus["industry"].unique())

print("\nConteo por industria:")
print(dataus["industry"].value_counts())

Industrias únicas:
['Trade' 'Software & Data' 'Food & Beverage']

Conteo por industria:
industry
Trade              3085
Software & Data    2056
Food & Beverage     834
Name: count, dtype: int64


PODEMOS OBSERVAR QUE PARA NUESTRA VARIABLE CATEGORIA EXISTEN 3 CATEGORIAS:

- TRADE
- SOFTWARE & DATA
- FOOD & BEVERAGE

## MODELO DE REGRESION

PARA EL MODELO INCLUIMOS LAS VARIABLES CATEGORIAS AHORA COMO VARIABLES DUMMY

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# Ordenar y calcular retorno log
dataus = dataus.sort_values(["firm", "year"]).copy()
dataus["price_lag"] = dataus.groupby("firm")["stockprice"].shift(1)
dataus["log_return"] = np.log(dataus["stockprice"]) - np.log(dataus["price_lag"])

# Filtrar
reg_df = dataus.dropna(subset=["log_return", "epsp", "industry"]).copy()

# Modelo OLS con interacción epsp × industria, tomando 'Trade' como referencia explícita
model = smf.ols(
    "log_return ~ epsp * C(industry, Treatment(reference='Trade'))",
    data=reg_df
).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             log_return   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     204.7
Date:                Thu, 16 Oct 2025   Prob (F-statistic):          2.86e-200
Time:                        19:00:01   Log-Likelihood:                -3717.1
No. Observations:                5349   AIC:                             7446.
Df Residuals:                    5343   BIC:                             7486.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

### PRIMERO DESCRIBAMOS LAS PREDICCIONES DE NUESTRO MODELO:

- INTERCEPT: ES EL INTERCEPTO (B0), EL NIVEL PROMEDIO CUANDO EPSP ES 0 (TOMANDO COMO INDUSTRIA BASE A TRADE)

- C(industry, Treatment(reference='Trade'))[T.Food & Beverage]: DE IGUAL MANERA ES EL INTERCEPTO CUANDO EPSP ES 0 PERO AHORA CON EL EFECTO DE LA VARIABLE CATEGORICA **FOOD & BEVERAGE**

- C(industry, Treatment(reference='Trade'))[T.Software & Data]: IGUAL QUE LA ANTERIOR ES EL INTERCEPTO CUANDO ESPS ES 0 CON EL EFECTO DE LA VARIABLE CATEGORICA **SOFTWARE & DATA**

- ESPS: EL EFECTO DE ESPS SOBRE EL RETORNO ANUAL LOGARITMICO(TOMANDO COMO INDUSTRIA BASE A TRADE)

- epsp:C(industry, Treatment(reference='Trade'))[T.Food & Beverage]: ES EL EFECTO SOBRE EL RETORNO ANUAL LOGARITMICO BAJO EL EFECTO DE LA VARIABLE CATEORICA **FOOD & BEVERAGE**

- epsp:C(industry, Treatment(reference='Trade'))[T.Software & Data]: ES EL EFECTO SOBRE EL RETORNO ANUAL LOGARITMICO BAJO EL EFECTO DE LA VARIABLE CATEORICA **SOFTWARE & DATA**

### VARIABLES ESTADISTICAMENTE SIGNIFICATIVAS

IDENTIFICAREMOS QUE VARIABLES SON ESTADISTICAMENTE SIGNIFICATIVAS O NO, DE ACUERDO A SU CI

- INTERCEPT: POSITIVAMENTE SIGNIFICATIVO

- C(industry, Treatment(reference='Trade'))[T.Food & Beverage]: NO SIGNIFICATIVO

- C(industry, Treatment(reference='Trade'))[T.Software & Data]: NO SIGNIFICATIVO

- ESPS: POSITIVAMENTE SIGNIFICATIVO

- epsp:C(industry, Treatment(reference='Trade'))[T.Food & Beverage]: POSITIVAMENTE SIGNIFICATIVO

- epsp:C(industry, Treatment(reference='Trade'))[T.Software & Data]: POSITIVAMENTE SIGNIFICATIVO

¿POR QUE?

OBSERVANDO SU CI LAS VARIABLES SIGNIFICATIVAS (QUE TODAS SON POSITIVAS) PODEMOS OBSERVAR QUE SUS VALORES DE CI SON POSITIVOS Y NO TIENEN VALOR 0.

LAS VARIABLES NO SIGNIFICATIVAS SU VALOR DE CI DIFIFIERE ENTRE VALORES NEGATIVOS Y POSITIVOS ENTONCES NO NOS PODEMOS VALER DE QUE VAYAN A TENER UN EFECTO POSITIVO O NEGATIVO.

IGUALMENTE PODEMOS CONFIRMARLO CON EL PVALUE YA QUE LAS VARIABLES ESTADISTICAMENTE SIGNIFICATICAS TIENEN UN VALOR MENOR DE 0.05

### COEFICIENTES

PODEMOS OBSERVAR QUE TODOS LOS COEFICENTES TIENEN UN EFECTO POSITIVO EN EL RETORNO ANUAL LOGARITMICO.

EXCLUYENDO A B0, PODEMOS OBSERVAR:

- LOS INTERCEPTOS EN ESPS = 0 CON EFECTO DE VARIABLES CATEGORICAS, SI BIEN, TIENEN UN VALOR POSITIVO, ADEMÁS DE QUE NO SON ESTADISTICAMENTE SIGNIFICATIVAS TIENEN UN EFECTO POSITIVO MUY PEQUEÑO

- EL ESPS AHORA SI, TIENE UN EFECTO POSTIVO SIGNIFICATIVO DE +0.1967 SOBRE EL RETORNO ANUAL LOGARITMICO, POR LO QUE PODEMOS DECIR QUE EFECTIVAMENTE LOS EARNINGS PER SHARE TIENEN UNA INCLUENCIA.

- PARA EL ESPS BAJO EL EFECTO DE LAS VARIABLES CATEGORICAS "FOOD & BEVERAGE" Y "SOFTWARE & DATA" TIENE CONSIDERABLEMENTE UN MAYOR EFECTO SOBRE EL COEFICIENTE BASE.


POR LO TANTO PODEMOS CONFIRMAR QUE EL ESPS NO SOLO TIENE UN EFECTO POSITIVO SI NO QUE DEPENDIENDO DE LA INDUSTRIA AUMENTA CONSIDERABLEMENTE.

## IDENTIFICANDO PROBLEMATIC OBSERVATIONS

AHORA IDENTIFICAREMOS OBSERVACIONES CON UN ALTO NIVEL DE LEVERAGE Y OUTLIERS, LOS QUITAREMOS DEL MODELO (SIENDO CONSERVADORES) Y NUEVAMENTE EJECUTAREMOS EL MODELO PARA SABER SI HAY UN CAMBIO EN LAS PREDICCIONES.

PARA IDENTIFICAR LOS OUTLIERS:

- SE IMPLEMENTARA LA METRICA COOK'S D, POR QUE INTEGRA EN UNA SOLA METRICA:
    - LEVERAGE: QUE TAN EXTREMO SE AJUSTA EN X
    - RESIDUO: QUE TAN MAL SE AJUSTA EN Y

- RETORNARE EL TOP 10 OBSERVACIONES POR COOK'S D


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# === 0) Partimos de 'reg_df' ya preparado ===
formula = "log_return ~ epsp * C(industry, Treatment(reference='Trade'))"

# === 1) Estimar el modelo original
m0 = smf.ols(formula, data=reg_df).fit()
print("\n=== MODELO ORIGINAL ===")
print(m0.summary())

# === 2) Medidas de influencia / outliers
infl = m0.get_influence()
leverage = infl.hat_matrix_diag                   # h_ii
resid_stud = infl.resid_studentized_external      # residuales studentizados
cooks_d = infl.cooks_distance[0]                  # Cook's D

n = m0.nobs
p = int(m0.df_model + 1)  # #parámetros incluyendo intercepto
lev_thresh = 3 * p / n    # regla: h_ii > 3*(k+1)/n
resid_thresh = 3          # |residual studentizado| > 3 (conservador)
cooks_thresh = 4 / n      # regla común: D_i > 4/n (también se usa >1 como muy estricto)

out = reg_df.copy()
out["_leverage"] = leverage
out["_rstud"] = resid_stud
out["_cooksD"] = cooks_d

out["_flag_lev"] = out["_leverage"] > lev_thresh
out["_flag_rstud"] = out["_rstud"].abs() > resid_thresh
out["_flag_cook"] = out["_cooksD"] > cooks_thresh

# Criterio conservador: marcar outlier si cumple al menos 2 criterios
out["_flags_sum"] = out[["_flag_lev","_flag_rstud","_flag_cook"]].sum(axis=1)
mask_outliers = out["_flags_sum"] >= 2

print("\n=== UMBRALES ===")
print(f"n={n}, p={p}, leverage_thr={lev_thresh:.4f}, |rstud|>{resid_thresh}, Cook'sD>{cooks_thresh:.6f}")
print(f"Outliers detectados (>=2 criterios): {mask_outliers.sum()} de {n}")

# (Opcional) inspeccionar top influyentes
top_infl = out.sort_values("_cooksD", ascending=False)[
    ["firm","year","industry","log_return","epsp","_leverage","_rstud","_cooksD","_flags_sum"]
].head(10)
print("\nTop 10 por Cook's D:")
print(top_infl.to_string(index=False))

# === 3) Recorte conservador y re-estimación
reg_df_trim = out.loc[~mask_outliers].drop(columns=[c for c in out.columns if c.startswith("_")])

m1 = smf.ols(formula, data=reg_df_trim).fit()
print("\n=== MODELO TRIMMED (sin outliers) ===")
print(m1.summary())

# === 4) Comparación lado a lado (coef, se, p)
def pack_results(m):
    return pd.DataFrame({
        "coef": m.params,
        "se": m.bse,
        "pval": m.pvalues
    })

comp = pack_results(m0).join(pack_results(m1), lsuffix="_orig", rsuffix="_trimmed")
comp["delta_coef"] = comp["coef_trimmed"] - comp["coef_orig"]
print("\n=== COMPARACIÓN COEFICIENTES (original vs trimmed) ===")
print(comp.to_string())

# (Opcional) versión con errores agrupados por firma para ambos modelos:
m0_cl = smf.ols(formula, data=reg_df).fit(cov_type="cluster", cov_kwds={"groups": reg_df["firm"]})
m1_cl = smf.ols(formula, data=reg_df_trim).fit(cov_type="cluster", cov_kwds={"groups": reg_df_trim["firm"]})
print("\n=== ORIGINAL (cluster por firm) ===")
print(m0_cl.summary())
print("\n=== TRIMMED (cluster por firm) ===")
print(m1_cl.summary())



=== MODELO ORIGINAL ===
                            OLS Regression Results                            
Dep. Variable:             log_return   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     204.7
Date:                Thu, 16 Oct 2025   Prob (F-statistic):          2.86e-200
Time:                        18:56:18   Log-Likelihood:                -3717.1
No. Observations:                5349   AIC:                             7446.
Df Residuals:                    5343   BIC:                             7486.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

### OBSERVACIONES NUEVO MODELO QUITANDO OUTLIERS

ES INTERESANTE POR QUE LAS VARIABLES SE MANTIENEN IGUAL EN CUANTO A SI SON ESTADISTICAMENTE SIGNIFICATIVAS O NO.


LOS QUE SI CAMBIO Y COMO ERA DE ESPERARSE SON LOS COEFICIENTES, LOS INTERCEPTOS SIN Y CON EFECTO DE UNA VARIABLE CATEGORICA AUNMENTARON POSITIVAMENTE CON EXEPCION DE C(industry, Treatment(reference='Trade'))[T.Software & Data] EL CUAL DISMINUYO.


EL ESPS SUBIO BRUTALMENTE DE 0.19 A 0.44 TENIENDO UN EFECTO MUCHO MAYOR EN EL RETORNO ANUAL LOGARITMICO


SOBRE EL VALOR DE ESPS BAJO EL EFECTO DE "FOOD & BEVEGARE" DISMINUYO CONSIDERABLEMENTE


FINALMENTE EL VALOR DE ESPS BAJO EL EFECTO DE "SOFTWARE & DATA" AUMENTO DE IGUAL MANERA RESPECTO A EL MODELO ORIGINAL

¿POR QUE?

ESTO ESTA TOTALMENTE POR CONSECUENCIA DE LOS OUTLIERS, SEGURAMENTE EN EL DATA SET ORIGINAL SIN HACER EL TRIM, HABIA OBSERVACIONES SUMAMENTE INFLUYENES Y AL QUITARLOS SE APLANA O DISMINUYE ESTE VALOR, SEGURAMENTE POR EMPRESAS GIGANTES DE ALIMENTOS COMO COCA COLA, PEPSI O ALGUNA OTRA QUE MANTIENEN CASI TODO EL MERCADO. IGUAL CONSIDERO QUE LAS EMPRESAS DE SOFTWARE TIENEN CONSERVAN UN VALOR SIMILAR DEBIDO A QUE LIDERAN LAS EMPRESAS DE ESTADOS UNIDOS EN VARIOS INDICES COMO SP500.